<a href="https://colab.research.google.com/github/dave-killough/databricks-colab/blob/main/PySpark%2C_Parquet%2C_Delta%2C_and_SQL_on_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Setup

In [ ]:
%pip install pyspark==3.5.0 delta-spark==3.0.0

In [ ]:
import requests
from pyspark.sql import SparkSession
from delta import *
import pyspark.sql
import os
import shutil

### Instantiate Spark and Delta

In [ ]:
builder = SparkSession.builder.appName("eo-master-file"
    ).config("spark.sql.extensions",
             "io.delta.sql.DeltaSparkSessionExtension"
    ).config("spark.sql.catalog.spark_catalog",
             "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    ).config("spark.jars.packages", "delta-spark:3.0.0")
spark = configure_spark_with_delta_pip(builder).getOrCreate()
# version that Databricks will use
#spark = SparkSession.builder.appName("eo-master-file").getOrCreate()

### Define Schema

In [ ]:
# IRS Exempt Organization Master File
# https://www.irs.gov/pub/irs-soi/eo_info.pdf
from pyspark.sql.types import StructType
from pyspark.sql.types import StructField
from pyspark.sql.types import StringType
from pyspark.sql.types import LongType
schema = StructType([
    StructField("EIN", StringType(), True),
    StructField("NAME", StringType(), True),
    StructField("ICO", StringType(), True),
    StructField("STREET", StringType(), True),
    StructField("CITY", StringType(), True),
    StructField("STATE", StringType(), True),
    StructField("ZIP", StringType(), True),
    StructField("RULING", StringType(), True),
    StructField("TAX_PERIOD", StringType(), True),
    StructField("GROUP", StringType(), True),
    StructField("SUBSECTION", StringType(), True),
    StructField("AFFILIATION", StringType(), True),
    StructField("CLASSIFICATION", StringType(), True),
    StructField("DEDUCTIBILITY", StringType(), True),
    StructField("FOUNDATION", StringType(), True),
    StructField("ACTIVITY", StringType(), True),
    StructField("ORGANIZATION", StringType(), True),
    StructField("STATUS", StringType(), True),
    StructField("ASSET_CD", StringType(), True),
    StructField("INCOME_CD", StringType(), True),
    StructField("FILING_REQ_CD", StringType(), True),
    StructField("PF_FILING_REQ_CD", StringType(), True),
    StructField("ACCT_PD", StringType(), True),
    StructField("ASSET_AMT", LongType(), True),
    StructField("INCOME_AMT", LongType(), True),
    StructField("REVENUE_AMT", LongType(), True),
    StructField("NTEE_CD", StringType(), True),
    StructField("SORT_NAME", StringType(), True)
])

### PySpark
https://spark.apache.org/docs/latest/api/python/index.html

In [ ]:
# ingest an example file
for eon in ['eo1','eo2','eo3','eo4']:
    response = requests.get(
        f"https://www.irs.gov/pub/irs-soi/{eon}.csv")
    with open(f"{eon}.csv", "wb") as f:
        f.write(response.content)

In [ ]:
# read from local csv file to spark dataframe
spark_csv_df = spark.read.csv(
    "eo[1234].csv",schema=schema, header=True)

In [ ]:
# display first 3 rows using pyspark
spark_csv_df.show(3)

In [ ]:
# display first 3 rows using pandas
spark_csv_df.limit(3).toPandas() # prettier display

### Parquet
https://parquet.apache.org/

In [ ]:
# transform to parquet - creates a directory
spark_csv_df.write.format("parquet"
    ).mode("overwrite").save("eo_parquet")

In [ ]:
# read delta table to dataframe
spark_parquet_df = spark.read.format("parquet"
    ).load("eo_parquet")
# display first 3 rows using pandas
spark_parquet_df.limit(3).toPandas()

In [ ]:
# transform to SQL
spark_parquet_df.createOrReplaceTempView('eo_parquet')
spark.sql("""
    SELECT NAME, CITY, STATE, ASSET_AMT
    FROM eo_parquet
    ORDER BY ASSET_AMT DESC LIMIT(20)
""").toPandas()

### Delta
https://delta.io/

In [ ]:
# transform to delta - creates a directory
if os.path.exists('eo_delta'):
    # overwrites cause growwwwwwth, so clear before
    if spark.catalog._jcatalog.tableExists("eo_delta"):
        spark.catalog.dropTempView("eo_delta")
    shutil.rmtree('eo_delta')
spark_csv_df.write.format("delta").save("eo_delta")

In [ ]:
# read delta table to dataframe
spark_delta_df = spark.read.format("delta").load("eo_delta")
spark_delta_df.limit(3).toPandas()

In [ ]:
# transform to SQL
spark_parquet_df.createOrReplaceTempView('eo_delta')
spark.sql("""
    SELECT NAME, CITY, STATE, ASSET_AMT
    FROM eo_delta
    ORDER BY ASSET_AMT DESC LIMIT(5)
""").toPandas()

### SQL

In [ ]:
spark.sql("SHOW TABLES").show()